In [7]:
import os

KAGGLE =  os.getenv("KAGGLE_URL_BASE") is not None
COLAB = os.getenv("COLAB_GPU") is not None
TPU = os.getenv("XRT_TPU_CONFIG") is not None
LOCAL = not KAGGLE and not COLAB

if not LOCAL:
    !git clone https://github.com/nclibz/MRKnee/

if COLAB:
    os.chdir('/content/MRKnee/')
    !git checkout v3
    from google.colab import drive
    drive.mount('/content/drive')
    DATADIR = "/content/drive/MyDrive/MRKnee/data"
    if TPU:
        !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

if KAGGLE:
    os.chdir('/kaggle/working/MRKnee/')
    !git checkout v3
    dataset_name = os.listdir('/kaggle/input')[0]
    
    DATADIR = f"/kaggle/input/{dataset_name}/MRNet"
    
    if TPU:
        !pip install torchtext==0.9
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py --version 1.8

if not LOCAL:
    !pip install --quiet "pytorch-lightning>=1.4.9" "torchmetrics>=0.5" "timm" "neptune-client" "optuna" "PyMySql" "torch-tb-profiler"
    !pip install albumentations --upgrade --quiet
    BACKBONE = "tf_efficientnetv2_s_in21k"

if LOCAL:
    DATADIR = "data"
    BACKBONE = 'tf_mobilenetv3_small_minimal_100'
    %load_ext autoreload
    %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
DIAGNOSIS = "acl"
PLANE = "sagittal"

In [9]:
from src.study import Study
from src.model import MRKnee
from src.data import MRKneeDataModule
from src.augmentations import Augmentations
from src.callbacks import Callbacks
from src.cfg import Cfg
import pytorch_lightning as pl

pl.seed_everything(123)

Global seed set to 123


123

In [10]:

def objective(trial, diagnosis=DIAGNOSIS, plane=PLANE, backbone=BACKBONE, datadir=DATADIR, profile = False):

    model = MRKnee(
        backbone=backbone,
        drop_rate=trial.suggest_int("drop_rate", 30, 80, step = 5) / 100,
        learning_rate=trial.suggest_loguniform('lr', 1e-6, 1e-3),
        adam_wd=trial.suggest_loguniform('adam_wd', 0.001, 0.1),
        max_epochs=20,
        precision=32,
        log_auc=True,
        log_ind_loss=False,

    )

    augs = Augmentations(
        model,
        max_res_train = 256,
        shift_limit=trial.suggest_int("shift_limit", 0, 15) / 100,
        scale_limit=trial.suggest_int("scale_limit", 0, 15) / 100,
        rotate_limit=trial.suggest_int("rotate_limit", 0, 15) / 100,
        reverse_p=0.0,
        indp_normalz=True,
    )

    dm = MRKneeDataModule(
        datadir=datadir,
        diagnosis=diagnosis,
        plane=plane,
        transforms=augs,
        clean=True,
        num_workers=2,
        pin_memory=True,
        trim_train=True,
    )

    configs = Cfg(model = model, dm = dm, augs = augs)
    cfg = configs.get_cfg()
    
    

    callbacks = Callbacks(cfg, trial, neptune_name="mrkneev3")
    neptune_logger = callbacks.get_neptune_logger()
    list_of_cbs = callbacks.get_callbacks()
    fast_dev_run = False
    
    if LOCAL:
        fast_dev_run = 50
    
    
    profiler = pl.profiler.PyTorchProfiler(dirpath = "src/logs", filename = "profiler") if profile else False
        


    trainer = pl.Trainer(
        gpus=1,
        precision=cfg["precision"],
        max_epochs=cfg["max_epochs"],
        logger=neptune_logger,
        log_every_n_steps=100,
        num_sanity_val_steps=0,
        callbacks=list_of_cbs,
        progress_bar_refresh_rate=20,
        deterministic=True,
        profiler = profiler, 
        fast_dev_run = fast_dev_run,
    )

    trainer.fit(model, dm)

    ## UPLOAD BEST CHECKPOINTS TO LOG
    if not LOCAL:
        callbacks.upload_best_checkpoints()

    return callbacks.model_checkpoint.best_model_score.item()


In [5]:
study = Study(diagnosis = DIAGNOSIS,
              plane = PLANE,
              backbone=BACKBONE,
              min_epochs=20) # no pruning

/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2021-10-12 10:38:22,457] A new study created in RDB with name: acl_sagittal_tf_mobilenetv3_small_minimal_100


In [12]:
study.optimize(objective, n_trials=3)

/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 50 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type        | Params
-----------------------------------------
0 | backbone | MobileNetV3 | 1.0 M 
1 | clf      | Linear      | 1.0 K 
2 | val_auc  | AUROC       | 0     
-----------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.082     Total estimated model params size (MB)
/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/pytorch_lightning/t

Epoch 0: 100%|██████████| 100/100 [00:05<00:00, 17.42it/s, loss=2.11, v_num=]

/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)


Epoch 0: 100%|██████████| 100/100 [00:08<00:00, 11.26it/s, loss=1.9, v_num=, val_auc=0.461, val_loss=1.160]
https://ui.neptune.ai/nclibz/mrkneev3/e/MRKNEEV-14


[W 2021-10-12 10:47:55,203] Trial 2 failed because of the following error: AttributeError("'NoneType' object has no attribute 'tolist'")
Traceback (most recent call last):
  File "/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_6183/2542897729.py", line 72, in objective
    callbacks.upload_best_checkpoints()
  File "/home/nicolai/OneDrive/Forskning/Projekter/MRKnee/src/callbacks.py", line 63, in upload_best_checkpoints
    "best_val_loss", self.model_checkpoint.best_model_score.tolist()
AttributeError: 'NoneType' object has no attribute 'tolist'


AttributeError: 'NoneType' object has no attribute 'tolist'